In [21]:
from transformers import pipeline
import fitz  # PyMuPDF

# 1) Load a FLAN-T5 text2text pipeline
#    (you can also use `AutoTokenizer` + `AutoModelForSeq2SeqLM` if you prefer)
extractor = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    tokenizer="google/flan-t5-small"
)

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n\n".join(page.get_text() for page in doc)

def extract_info_via_t5(text: str):
    # define the slots you want to extract
    questions = {
        "company name":      "Extract the company name",
        "focus":             "Extract the company's main focus",
        "services":          "Extract the company's core services"
    }
    results = {}
    for key, instruction in questions.items():
        prompt = (
            f"{instruction} from the following text:\n\n{text}\n\n"
            "Answer concisely."
        )
        out = extractor(
            prompt,
            max_new_tokens=64,
            do_sample=False
        )
        # the pipeline returns a list of dicts; grab the generated text
        results[key] = out[0]["generated_text"].strip()
    return results


pdf_text = extract_text_from_pdf("./E-servises .pdf")
info = extract_info_via_t5(pdf_text)
print(info)
    # → e.g. {'company name': 'Acme Corp.', 'focus': 'manufacturing widgets', ...}


Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


{'company name': 'AfriSwitch Technologies Ltd', 'focus': '1. Company Registration: AfriSwitch Technologies Ltd 2. Shareholding: o 60% Tech Partners Ltd 3. Technical Proof: o Patent-pending switching algorithm o MTN USSD test environment', 'services': '1. Company Registration: AfriSwitch Technologies Ltd 2. Shareholding: o 60% Tech Partners Ltd 3. Technical Proof: o Patent-pending switching algorithm o MTN USSD test environment'}
